In [1]:
import optuna
import joblib
import datetime

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from stesml.model_tools import build_train_test_model

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
def objective(trial):   
    # Save the study before running the next trial
    joblib.dump(study, "../studies/study_" + model_type + "_" + datetime.datetime.now().strftime("%Y%m%d-%H") + ".pkl")
    
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        parameters = {'n_layers': n_layers, 'n_hidden_units': n_hidden_units, 'batch_size': batch_size, 'epochs': 200}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_max=360)
    elif model_type == 'XGBoost':
        scale = False
        learning_rate = trial.suggest_float("learning_rate", 0.01, 1)
        subsample = 1
        colsample_bytree = 1
        #subsample = trial.suggest_float("subsample", 0.01, 1)
        #colsample_bytree = trial.suggest_float("colsample_bytree", 0.01, 1)
        num_boost_round = 10000 # Set this as a maximum, model will stop with early stopping
        parameters = {'learning_rate': learning_rate, 'subsample': subsample, 'colsample_bytree': colsample_bytree, 'num_boost_round': num_boost_round}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats, t_min=360)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 200, log=True)
        max_depth = trial.suggest_int("max_depth", 1, 100)
        max_samples = trial.suggest_float("max_samples", .01, 1, log=True)
        parameters = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_samples': max_samples}
        result, addendum = build_train_test_model(data_dir, model_type, target, metric, scale, parameters, n_repeats)
    
    return result

In [4]:
data_dir = "../data/Sulfur_Models/heating/heating_all"
model_type = 'XGBoost' # Options: NN, XGBoost, RandomForest
target = 'h' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_repeats = 1 # Number of times to repeat 5-fold CV. Each repeat gives a different shuffle.

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'
    
load_study = False
study_name = "study_XGBoost_20220708-19.pkl"

In [5]:
if load_study:
    study = joblib.load("../studies/" + study_name)
    print("Best trial until now:")
    print(" Value: ", study.best_trial.value)
    print(" Params: ")
    for key, value in study.best_trial.params.items():
        print(f"    {key}: {value}")
else:
    study = optuna.create_study(direction=direction)

[I 2022-07-11 16:14:27,372] A new study created in memory with name: no-name-53040c49-01e5-47d0-b671-f669a356525a


In [ ]:
study.optimize(objective, n_trials=250)

[16:14:42] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:9.73112
[20]	test-rmse:7.25621
[40]	test-rmse:7.16270
[48]	test-rmse:7.16929
Split #0, This Result: 7.1689, Average Result: 7.1689
[16:15:34] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:11.04895
[20]	test-rmse:6.49078
[23]	test-rmse:6.49839
Split #1, This Result: 6.4984, Average Result: 6.8337


[I 2022-07-11 16:18:14,680] Trial 0 finished with value: 5.5802088843467414 and parameters: {'learning_rate': 0.8458294256433071}. Best is trial 0 with value: 5.5802088843467414.


Split #4, This Result: 7.8062, Average Result: 5.5802
[16:18:27] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:8.10350
[20]	test-rmse:6.64657
[35]	test-rmse:6.66178
Split #0, This Result: 6.6618, Average Result: 6.6618
[16:19:08] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:8.35273
[20]	test-rmse:6.44799
[40]	test-rmse:6.44415
[56]	test-rmse:6.44634
S

[I 2022-07-11 16:21:18,514] Trial 1 finished with value: 4.998480093756328 and parameters: {'learning_rate': 0.9402138899698731}. Best is trial 1 with value: 4.998480093756328.


Split #4, This Result: 4.2432, Average Result: 4.9985
[16:21:30] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	test-rmse:44.00529
[20]	test-rmse:23.69805
[40]	test-rmse:13.46896
[60]	test-rmse:9.14966
[80]	test-rmse:7.82750
[100]	test-rmse:7.41089
[120]	test-rmse:7.41480
[125]	test-rmse:7.41840
Split #0, This Result: 7.4207, Average Result: 7.4207
[16:23:11] WARNING: ../src/learner.cc:576: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue i

In [ ]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621